<a href="https://colab.research.google.com/github/tremblay17/Machine-Learning-Projects/blob/main/StockClassifier-Keras-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install yfinance
%pip install pandas_ta
%pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.model_selection import train_test_split
import keras as k
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.python.framework.ops import default_session
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.metrics import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris, load_digits
import yfinance as yf
import pandas_ta as ta
import optuna as o

In [ ]:
tf.test.gpu_device_name()
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11066356821046948268
xla_global_id: -1
]


In [ ]:
np.set_printoptions(precision=3, suppress=True)

startDate = "2022-12-06" #@param {type:"date"}
endDate = "2022-12-06" #@param {type: "date"}
stock = 'NVDA' #@param {type: "string"}
period = '10y' #@param ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y']
interval= '1d' #@param ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']
toDate_historical = 'toDate' #@param ['toDate', 'historical']
length_days = 14 #@param {type:"slider", min:1, max:200, step:1}
macdFast = 12 #@param {type:"slider", min:1, max:200, step:1}
macdSlow = 26 #@param {type:"slider", min:1, max:200, step:1}
macdSignal = 9 #@param {type:"slider", min:1, max:200, step:1}
stochFastK = 14 #@param {type:"slider", min:1, max:200, step:1}
stochSlowK = 3 #@param {type:"slider", min:1, max:200, step:1}
stochD = 3 #@param {type:"slider", min:1, max:200, step:1}

if(toDate_historical == 'toDate'):
  data = yf.Ticker(stock).history(period=period, interval=interval)[
      ['Close', 'Open', 'High', 'Volume', 'Low']]
else:
  data = yf.Ticker(stock).history(start=startDate, end=endDate, interval=interval)[
      ['Close', 'Open', 'High', 'Volume', 'Low']]

data.ta.macd(close='close', fast=macdFast, slow=macdSlow, signal=macdSignal, append=True)
data.ta.adx(length=length_days, append=True)
data.ta.rsi(length=length_days, append=True)
data.ta.bbands(length=length_days, append=True)
data.ta.obv(append=True)
data.ta.stoch(high='High', low='Low', fast_k=stochFastK, slow_k=stochSlowK, slow_d=stochD, append=True)
data.ta.aroon(length=length_days, append=True)

print(data.info())

Class_Cat = []
Class_Int = []
for macd, macdh, macds, adx, dmp, dmn, rsi, bbl, bbm, bbu, bbb, bbp, obv, stochk, stochd, aroond, aroonu, aroonosc in data.iloc[:, 5:].values:
  if ((macd == 'NaN') or (macdh == 'NaN') or (macds == 'NaN') or (adx == 'NaN') or (dmp == 'NaN') or (dmn == 'NaN') or (rsi == 'NaN')):
    Class_Cat.append('Hold')
    Class_Int.append(1)
  elif (((macd >= macds) and (macdh > 0) and ((adx >= 25) and (dmp > dmn))) or (rsi <= 30)):
    Class_Cat.append('Buy')
    Class_Int.append(0)
  elif (((macd <= macds) and (macdh < 0) and ((adx >= 25) and (dmn > dmp))) or (rsi >= 70)):
    Class_Cat.append('Sell')
    Class_Int.append(2)
  else:
    Class_Cat.append('Hold')
    Class_Int.append(1)

data['Class_Int'] = Class_Int
data['Class_Cat'] = Class_Cat
data = data.iloc[34:, :]
data.head(60)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2012-12-10 00:00:00-05:00 to 2022-12-08 00:00:00-05:00
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close          2518 non-null   float64
 1   Open           2518 non-null   float64
 2   High           2518 non-null   float64
 3   Volume         2518 non-null   int64  
 4   Low            2518 non-null   float64
 5   MACD_12_26_9   2493 non-null   float64
 6   MACDh_12_26_9  2485 non-null   float64
 7   MACDs_12_26_9  2485 non-null   float64
 8   ADX_14         2491 non-null   float64
 9   DMP_14         2504 non-null   float64
 10  DMN_14         2504 non-null   float64
 11  RSI_14         2504 non-null   float64
 12  BBL_14_2.0     2505 non-null   float64
 13  BBM_14_2.0     2505 non-null   float64
 14  BBU_14_2.0     2505 non-null   float64
 15  BBB_14_2.0     2505 non-null   float64
 16  BBP_14_2.0     2505 non-null   float64
 17  OBV 

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,...,BBB_14_2.0,BBP_14_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3,AROOND_14,AROONU_14,AROONOSC_14,Class_Int,Class_Cat
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-30 00:00:00-05:00,2.844914,2.851841,2.891098,28759200,2.835677,-0.011685,0.010640,-0.022325,13.613088,23.518523,...,4.869210,0.648717,77604000.0,60.987114,54.360233,28.571429,85.714286,57.142857,1,Hold
2013-01-31 00:00:00-05:00,2.831058,2.844913,2.877242,47054400,2.812585,-0.011810,0.008412,-0.020222,12.782976,21.674450,...,4.874235,0.544319,30549600.0,50.854670,56.143603,21.428571,78.571429,57.142857,1,Hold
2013-02-01 00:00:00-05:00,2.856460,2.854151,2.865697,28881600,2.831059,-0.009747,0.008380,-0.018127,12.039533,20.736398,...,4.995080,0.704368,59431200.0,52.136715,54.659499,14.285714,71.428571,57.142857,1,Hold
2013-02-04 00:00:00-05:00,2.807967,2.840296,2.898025,51174400,2.805658,-0.011888,0.004992,-0.016879,12.182700,22.313229,...,5.034903,0.366736,8256800.0,45.299158,49.430181,7.142857,64.285714,57.142857,1,Hold
2013-02-05 00:00:00-05:00,2.872623,2.821822,2.888788,36729600,2.812585,-0.008272,0.006886,-0.015158,12.311691,20.208172,...,4.671761,0.787724,44986400.0,52.259730,49.898534,0.000000,57.142857,57.142857,1,Hold
2013-02-06 00:00:00-05:00,2.849532,2.803349,2.872624,53068800,2.787184,-0.007186,0.006377,-0.013563,11.644653,18.248226,...,4.376700,0.587625,-8082400.0,49.026753,48.861880,0.000000,50.000000,50.000000,1,Hold
2013-02-07 00:00:00-05:00,2.837986,2.840295,2.863387,44000000,2.810276,-0.007175,0.005110,-0.012286,11.040732,17.135654,...,4.358869,0.489339,-52082400.0,52.352621,51.213035,21.428571,42.857143,21.428571,1,Hold
2013-02-08 00:00:00-05:00,2.856460,2.856460,2.879552,30723200,2.837986,-0.005611,0.005340,-0.010951,10.940954,18.198850,...,4.238446,0.614999,-21359200.0,48.309317,49.896230,14.285714,35.714286,21.428571,1,Hold
2013-02-11 00:00:00-05:00,2.888788,2.879551,2.918807,40830800,2.851841,-0.001743,0.007366,-0.009109,11.801225,21.358543,...,4.125941,0.836739,19471600.0,56.521798,52.394579,7.142857,28.571429,21.428571,1,Hold


In [ ]:
data_inputs = data.iloc[:, 0:23].values
data_class = data.iloc[:, 23:24].values
print(data_class)


[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [ ]:
# Classes 'Buy(1)', 'Hold(0)', or 'Sell(2)'
np.asarray(data_inputs).astype('float64')
np.asarray(data_class).astype('float64')


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [ ]:

sc = MinMaxScaler(feature_range=(0, 1))
x_train, x_test, y_train, y_test = train_test_split(
    data_inputs, data_class, train_size=0.75)


In [ ]:
# Normalization

sc.fit(x_train)
sc.transform(x_train)
sc.transform(x_test)

x_train.shape


(1863, 23)

In [ ]:

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape


(1863, 23, 1)

In [ ]:
 #Currently generates the highest accuracy
#{'layer1': 75, 'activation1': 'tanh',
#'layer2': 85, 'activation2': 'softsign',
#'layer3': 73, 'activation3': 'softsign',
#'layer4': 29, 'activation4': 'relu',
#'learning rate': 0.006548463552532551}

layerSize_1 = 75 #@param {type:"slider", min:1, max:200, step:1}
layerSize_2 = 85 #@param {type:"slider", min:1, max:200, step:1}
layerSize_3 = 73 #@param {type:"slider", min:1, max:200, step:1}
layerSize_4 = 29 #@param {type:"slider", min:1, max:200, step:1}

seqModel = Sequential(
    [
        LSTM(units=layerSize_1, return_sequences=True, activation='tanh',
             input_shape=(x_train.shape[1], 1)),

        LSTM(units=layerSize_2, return_sequences=True, activation='softsign'),

        LSTM(units=layerSize_3, return_sequences=True, activation='softsign'),

        LSTM(units=layerSize_4, activation='relu'),

        Dense(units=3, activation='sigmoid')
    ]
)
#Adding a 5th layer led to Diminishing Returns


In [ ]:
#@title Default title text
from keras.losses import *

lossFunc = SparseCategoricalCrossentropy() #@param ["SparseCategoricalCrossentropy()", "Poisson()", "KLDivergence()"] {type:"raw"}
batchSize = 64  #@param {type:"slider", min:1, max:200, step:1}
epochs = 75  #@param {type:"slider", min:1, max:200, step:1}
optimizer = Adam(learning_rate=0.006548463552532551) #@param ["Adam(learning_rate=0.006548463552532551)", "Adadelta()", "Adamax()", "Nadam()", 'SGD()'] {type:"raw"}
metrics = [SparseCategoricalAccuracy(), MeanAbsoluteError(), MeanSquaredError(), CategoricalHinge(), Poisson(), KLDivergence()]
seqModel.compile(optimizer=optimizer, loss=lossFunc, metrics=metrics)


In [ ]:
seqModel.fit(x_train,
             y_train,
             validation_data=(x_test, y_test),
             epochs=epochs, batch_size=batchSize, validation_batch_size=batchSize)

Epoch 1/75
30/30 [==============================] - 20s 325ms/step - loss: 0.8826 - sparse_categorical_accuracy: 0.6479 - mean_absolute_error: 0.5656 - mean_squared_error: 0.4483 - categorical_hinge: 0.3625 - poisson: 1.0983 - kl_divergence: 1.2894 - val_loss: 0.8330 - val_sparse_categorical_accuracy: 0.6844 - val_mean_absolute_error: 0.5910 - val_mean_squared_error: 0.4751 - val_categorical_hinge: 0.3314 - val_poisson: 1.1322 - val_kl_divergence: 1.4951
Epoch 2/75
30/30 [==============================] - 4s 121ms/step - loss: 0.7554 - sparse_categorical_accuracy: 0.6758 - mean_absolute_error: 0.5779 - mean_squared_error: 0.4846 - categorical_hinge: 0.3596 - poisson: 1.1303 - kl_divergence: 1.3904 - val_loss: 0.7861 - val_sparse_categorical_accuracy: 0.5942 - val_mean_absolute_error: 0.5495 - val_mean_squared_error: 0.4740 - val_categorical_hinge: 0.3540 - val_poisson: 1.1496 - val_kl_divergence: 1.3680
Epoch 3/75
30/30 [==============================] - 3s 109ms/step - loss: 0.6548 - 

In [ ]:
print(seqModel.evaluate(x_train, y_train))
seqModel.evaluate(x_test, y_test)

59/59 [==============================] - 1s 21ms/step - loss: 0.1501 - sparse_categorical_accuracy: 0.9431 - mean_absolute_error: 0.6668 - mean_squared_error: 0.7219 - categorical_hinge: 0.4016 - poisson: 2.7172 - kl_divergence: 6.1180
[0.15014289319515228, 0.9431025385856628, 0.6668065190315247, 0.7218831777572632, 0.4015952944755554, 2.7171807289123535, 6.117966651916504]
20/20 [==============================] - 0s 21ms/step - loss: 0.4677 - sparse_categorical_accuracy: 0.8937 - mean_absolute_error: 0.6715 - mean_squared_error: 0.7299 - categorical_hinge: 0.3774 - poisson: 2.7567 - kl_divergence: 6.3225


[0.4676613509654999,
 0.8937197923660278,
 0.6714918613433838,
 0.7299188375473022,
 0.37735414505004883,
 2.756727457046509,
 6.322474479675293]

In [ ]:
print(x_test)
pred = seqModel.predict(x_train)

[[[  3.722]
  [  3.725]
  [  3.753]
  ...
  [ 28.571]
  [ 92.857]
  [ 64.286]]

 [[301.698]
  [311.209]
  [312.758]
  ...
  [ 71.429]
  [  7.143]
  [-64.286]]

 [[  7.247]
  [  7.242]
  [  7.298]
  ...
  [100.   ]
  [ 50.   ]
  [-50.   ]]

 ...

 [[121.389]
  [124.593]
  [126.259]
  ...
  [ 78.571]
  [ 57.143]
  [-21.429]]

 [[ 23.212]
  [ 23.276]
  [ 23.444]
  ...
  [  0.   ]
  [ 92.857]
  [ 92.857]]

 [[134.576]
  [137.234]
  [137.714]
  ...
  [ 85.714]
  [  0.   ]
  [-85.714]]]
59/59 [==============================] - 2s 20ms/step


In [ ]:
np.set_printoptions(precision=3, suppress=True)

stock = 'GOOGL'

if(toDate_historical == 'toDate'):
  data = yf.Ticker(stock).history(period=period, interval=interval)[
      ['Close', 'Open', 'High', 'Volume', 'Low']]
else:
  data = yf.Ticker(stock).history(start=startDate, end=endDate, interval=interval)[
      ['Close', 'Open', 'High', 'Volume', 'Low']]

data.ta.macd(close='close', fast=macdFast, slow=macdSlow, signal=macdSignal, append=True)
data.ta.adx(length=length_days, append=True)
data.ta.rsi(length=length_days, append=True)
data.ta.bbands(length=length_days, append=True)
data.ta.obv(append=True)
data.ta.stoch(high='High', low='Low', fast_k=stochFastK, slow_k=stochSlowK, slow_d=stochD, append=True)
data.ta.aroon(length=length_days, append=True)

print(data.info())

Class_Cat = []
Class_Int = []
for macd, macdh, macds, adx, dmp, dmn, rsi, bbl, bbm, bbu, bbb, bbp, obv, stochk, stochd, aroond, aroonu, aroonosc in data.iloc[:, 5:].values:
  if ((macd == 'NaN') or (macdh == 'NaN') or (macds == 'NaN') or (adx == 'NaN') or (dmp == 'NaN') or (dmn == 'NaN') or (rsi == 'NaN')):
    Class_Cat.append('Hold')
    Class_Int.append(1)
  elif (((macd >= macds) and (macdh > 0) and ((adx >= 25) and (dmp > dmn))) or (rsi <= 30)):
    Class_Cat.append('Buy')
    Class_Int.append(0)
  elif (((macd <= macds) and (macdh < 0) and ((adx >= 25) and (dmn > dmp))) or (rsi >= 70)):
    Class_Cat.append('Sell')
    Class_Int.append(2)
  else:
    Class_Cat.append('Hold')
    Class_Int.append(1)

data['Class_Int'] = Class_Int
data['Class_Cat'] = Class_Cat
data = data.iloc[34:, :]

data_inputs = data.iloc[:, 0:23].values
data_class = data.iloc[:, 23:24].values
print(data_class)

# Classes 'Buy(1)', 'Hold(0)', or 'Sell(2)'
np.asarray(data_inputs).astype('float64')
np.asarray(data_class).astype('float64')
sc = MinMaxScaler(feature_range=(0, 1))
x_train, x_test, y_train, y_test = train_test_split(
    data_inputs, data_class, train_size=0.5)
# Normalization
sc.fit(x_train)
sc.transform(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


print(data.head(5))
data.tail(5)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2012-12-10 00:00:00-05:00 to 2022-12-08 00:00:00-05:00
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close          2518 non-null   float64
 1   Open           2518 non-null   float64
 2   High           2518 non-null   float64
 3   Volume         2518 non-null   int64  
 4   Low            2518 non-null   float64
 5   MACD_12_26_9   2493 non-null   float64
 6   MACDh_12_26_9  2485 non-null   float64
 7   MACDs_12_26_9  2485 non-null   float64
 8   ADX_14         2491 non-null   float64
 9   DMP_14         2504 non-null   float64
 10  DMN_14         2504 non-null   float64
 11  RSI_14         2504 non-null   float64
 12  BBL_14_2.0     2505 non-null   float64
 13  BBM_14_2.0     2505 non-null   float64
 14  BBU_14_2.0     2505 non-null   float64
 15  BBB_14_2.0     2505 non-null   float64
 16  BBP_14_2.0     2505 non-null   float64
 17  OBV 

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,...,BBB_14_2.0,BBP_14_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3,AROOND_14,AROONU_14,AROONOSC_14,Class_Int,Class_Cat
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-02 00:00:00-05:00,100.440002,99.050003,100.769997,21469400,98.900002,0.960859,0.760079,0.200779,15.848050,30.030002,...,7.664025,0.834525,5.136061e+09,87.423508,81.921898,0.000000,92.857143,92.857143,1,Hold
2022-12-05 00:00:00-05:00,99.480003,99.400002,101.379997,24405100,99.000000,1.013695,0.650332,0.363363,15.398250,29.779237,...,7.359703,0.677209,5.111656e+09,76.340106,81.904788,42.857143,85.714286,42.857143,1,Hold
2022-12-06 00:00:00-05:00,96.980003,99.300003,99.779999,24910700,96.419998,0.844108,0.384597,0.459512,14.469444,27.404516,...,7.469703,0.347832,5.086745e+09,59.959372,74.574329,35.714286,78.571429,42.857143,1,Hold
2022-12-07 00:00:00-05:00,94.940002,96.410004,96.879997,31045400,94.720001,0.538887,0.063500,0.475387,14.107694,25.907982,...,8.125273,0.138175,5.055700e+09,37.601640,57.967039,28.571429,71.428571,42.857143,1,Hold
2022-12-08 00:00:00-05:00,93.709999,95.379997,95.580002,32185900,93.449997,0.195492,-0.223916,0.419408,14.125266,24.547394,...,9.193993,0.078966,5.023514e+09,16.513316,38.024776,100.000000,64.285714,-35.714286,1,Hold


In [ ]:
print(seqModel.predict(x_test))


39/39 [==============================] - 1s 21ms/step
[[0.004 0.969 0.949]
 [0.001 1.    0.065]
 [0.004 0.489 0.991]
 ...
 [0.001 1.    0.278]
 [0.02  0.999 0.096]
 [0.    1.    0.186]]
